# -----------------------------------------Read Me----------------------------------------------------------

###### The start part includes allot of testing and checking the data and trying to find the best solution.

###### this is very important and it shows and examples and way to accomplish the final goal which is to enter a list of the companies tickers you wish to get thier SEC Filing Reports and get the States Of Income statment and evantually do as you wish with this data.

###### in the end of the page there is the final code.

Link to a reference walktrough explanation - https://sec-api.io/resources/extract-financial-statements-from-sec-filings-and-xbrl-data-with-python

# ---------------------------------------------------------------------------------------------------------------

In [112]:
from sec_api import XbrlApi
import requests

In [2]:
API_KEY = "6ade3b2f0a2495cf4324e580c9ca163f890a2f2d47599ec28ff57bfd4c41cf2e"

xbrlApi = XbrlApi(API_KEY)

In [3]:
# URL of Apple's 10-K filings
url_10k_aapl = 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm'
aapl_xbrl_json = xbrlApi.xbrl_to_json(htm_url=url_10k_aapl)

In [ ]:
print("Keys of income statement dictionary in XBRL from Apple's 10-K filing (sorted)")
print('-----------------------------------------------------------------------------')
keys = list(aapl_xbrl_json.keys())
keys.sort()  # Sort the keys alphabetically
print(*keys, sep='\n')

In [ ]:
print("Keys of income statement dictionary in XBRL from Apple's 10-K filing")
print('--------------------------------------------------------------------')
keys = list(aapl_xbrl_json['StatementsOfIncome'].keys())
keys.sort()  # Sort the keys alphabetically
print(*keys, sep='\n')

In [41]:
import pandas as pd 

# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + ' to ' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 


income_statement_aapl = get_income_statement(aapl_xbrl_json)


print("Income statement of Apple's 2022 10-K filing as dataframe")
print('---------------------------------------------------------')
income_statement_aapl.head(10)

Income statement of Apple's 2022 10-K filing as dataframe
---------------------------------------------------------


,2021-09-26 to 2022-09-24,2020-09-27 to 2021-09-25,2019-09-29 to 2020-09-26
RevenueFromContractWithCustomerExcludingAssessedTax,394328000000,365817000000,274515000000
CostOfGoodsAndServicesSold,223546000000,212981000000,169559000000
GrossProfit,170782000000,152836000000,104956000000
ResearchAndDevelopmentExpense,26251000000,21914000000,18752000000
SellingGeneralAndAdministrativeExpense,25094000000,21973000000,19916000000
OperatingExpenses,51345000000,43887000000,38668000000
OperatingIncomeLoss,119437000000,108949000000,66288000000
NonoperatingIncomeExpense,-334000000,258000000,803000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,119103000000,109207000000,67091000000
IncomeTaxExpenseBenefit,19300000000,14527000000,9680000000


# Change the index into 10-k syntex

In [42]:
def change_index_human_readable(income_statement):
    
    human_readable_filing = {'RevenueFromContractWithCustomerExcludingAssessedTax': 'Total net sales',
     'CostOfGoodsAndServicesSold': 'Total cost of sales',
     'GrossProfit': 'Gross margin',
     'ResearchAndDevelopmentExpense': 'Research and development',
     'SellingGeneralAndAdministrativeExpense': 'Selling, general and administrative',
     'OperatingExpenses': 'Total operating expenses',
     'OperatingIncomeLoss': 'Operating income',
     'NonoperatingIncomeExpense': 'Other income/(expense), net',
     'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest': 'Income before provision for income taxes',
     'IncomeTaxExpenseBenefit': 'Provision for income taxes',
     'NetIncomeLoss': 'Net income',
     'EarningsPerShareBasic': 'Earnings per share: Basic',
     'EarningsPerShareDiluted': 'Earnings per share: Diluted',
     'WeightedAverageNumberOfSharesOutstandingBasic': 'Shares used in computing earnings per share: Basic',
     'WeightedAverageNumberOfDilutedSharesOutstanding': 'Shares used in computing earnings per share: Diluted'}

    for index, row in income_statement.iterrows():
        desired_index = index
        new_index_name = human_readable_filing[index]

        income_statement = income_statement.rename(index={desired_index: new_index_name})
        
    return income_statement
    
income_statement_aapl = change_index_human_readable(income_statement_aapl)

In [43]:
income_statement_aapl.head(5)

,2021-09-26 to 2022-09-24,2020-09-27 to 2021-09-25,2019-09-29 to 2020-09-26
Total net sales,394328000000,365817000000,274515000000
Total cost of sales,223546000000,212981000000,169559000000
Gross margin,170782000000,152836000000,104956000000
Research and development,26251000000,21914000000,18752000000
"Selling, general and administrative",25094000000,21973000000,19916000000
Total operating expenses,51345000000,43887000000,38668000000
Operating income,119437000000,108949000000,66288000000
"Other income/(expense), net",-334000000,258000000,803000000
Income before provision for income taxes,119103000000,109207000000,67091000000
Provision for income taxes,19300000000,14527000000,9680000000


# Change the column date labels into 10-k format (more easy to read)

In [108]:
def change_date_column_format(income_statement):
    
    date_lst = []
    for i in income_statement.columns:
        temp = i.split()
        start = pd.to_datetime(temp[0]).strftime("%B %d, %Y")
        end = pd.to_datetime(temp[2]).strftime("%B %d, %Y")
        date_lst.append([start, end])
    
    for i in range(len(income_statement.columns)):
        end_date = date_lst[i][1]
        end_date_str = end_date[0] if isinstance(end_date, list) else end_date
        income_statement = income_statement.rename(columns={income_statement.columns[i]: f'Years ended - {end_date_str}'})
    
    return income_statement

In [ ]:
income_statement_aapl = change_date_column_format(income_statement_aapl)

In [48]:
income_statement_aapl.head(5)

,"Years ended - September 24, 2022","Years ended - September 25, 2021","Years ended - September 26, 2020"
Total net sales,394328000000,365817000000,274515000000
Total cost of sales,223546000000,212981000000,169559000000
Gross margin,170782000000,152836000000,104956000000
Research and development,26251000000,21914000000,18752000000
"Selling, general and administrative",25094000000,21973000000,19916000000
Total operating expenses,51345000000,43887000000,38668000000
Operating income,119437000000,108949000000,66288000000
"Other income/(expense), net",-334000000,258000000,803000000
Income before provision for income taxes,119103000000,109207000000,67091000000
Provision for income taxes,19300000000,14527000000,9680000000


# Extract and Merge Financial Statements from Multiple 10-K Filings

In [58]:
# Apple's 10Ks of the last 4 years, 2019 to 2022
url_10k_2019 = 'https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/a10-k20199282019.htm'
url_10k_2020 = 'https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/aapl-20200926.htm'
url_10k_2021 = 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000105/aapl-20210925.htm'
url_10k_2022 = 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm'

xbrl_json_2019 = xbrlApi.xbrl_to_json(htm_url=url_10k_2019)
xbrl_json_2020 = xbrlApi.xbrl_to_json(htm_url=url_10k_2020)
xbrl_json_2021 = xbrlApi.xbrl_to_json(htm_url=url_10k_2021)
xbrl_json_2022 = xbrlApi.xbrl_to_json(htm_url=url_10k_2022)

In [282]:
# Calling the get_income_statement we defiend at the start
income_statement_2019 = get_income_statement(xbrl_json_2019)
income_statement_2020 = get_income_statement(xbrl_json_2020)
income_statement_2021 = get_income_statement(xbrl_json_2021)
income_statement_2022 = get_income_statement(xbrl_json_2022)

In [283]:
income_statements_merged = pd.concat([income_statement_2019, income_statement_2020, income_statement_2021, income_statement_2022], axis=0, sort=False)

# sort & reset the index of the merged dataframe
income_statements_merged = income_statements_merged.sort_index().reset_index()

# convert cells to float
income_statements_merged = income_statements_merged.applymap(lambda x: pd.to_numeric(x, errors='ignore'))


print("Merged, uncleaned financials of all income statements")
print('-----------------------------------------------------')
income_statements_merged.head(5)

Merged, uncleaned financials of all income statements
-----------------------------------------------------


,index,2016-09-25 to 2017-09-30,2017-10-01 to 2017-12-30,2017-10-01 to 2018-09-29,2017-12-31 to 2018-03-31,2018-04-01 to 2018-06-30,2018-07-01 to 2018-09-29,2018-09-30 to 2018-12-29,2018-09-30 to 2019-09-28,2018-12-30 to 2019-03-30,2019-03-31 to 2019-06-29,2019-06-30 to 2019-09-28,2019-09-29 to 2019-12-28,2019-09-29 to 2020-09-26,2019-12-29 to 2020-03-28,2020-03-29 to 2020-06-27,2020-06-28 to 2020-09-26,2020-09-27 to 2021-09-25,2021-09-26 to 2022-09-24
0,CostOfGoodsAndServicesSold,1.410480e+11,NaN,1.637560e+11,NaN,NaN,NaN,NaN,1.617820e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CostOfGoodsAndServicesSold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.695590e+11,NaN,NaN,NaN,2.129810e+11,2.235460e+11
2,CostOfGoodsAndServicesSold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.617820e+11,NaN,NaN,NaN,NaN,1.695590e+11,NaN,NaN,NaN,2.129810e+11,NaN
3,CostOfGoodsAndServicesSold,NaN,NaN,1.637560e+11,NaN,NaN,NaN,NaN,1.617820e+11,NaN,NaN,NaN,NaN,1.695590e+11,NaN,NaN,NaN,NaN,NaN
4,EarningsPerShareBasic,NaN,NaN,3.000000e+00,NaN,NaN,NaN,1.05,2.990000e+00,0.62,0.55,0.76,1.26,3.310000e+00,0.64,0.65,0.74,NaN,NaN
5,EarningsPerShareBasic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.990000e+00,NaN,NaN,NaN,NaN,3.310000e+00,NaN,NaN,NaN,5.670000e+00,NaN
6,EarningsPerShareBasic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.310000e+00,NaN,NaN,NaN,5.670000e+00,6.150000e+00
7,EarningsPerShareBasic,9.270000e+00,3.92,1.201000e+01,2.75,2.36,2.94,4.22,1.197000e+01,2.47,2.20,3.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,EarningsPerShareDiluted,9.210000e+00,3.89,1.191000e+01,2.73,2.34,2.91,4.18,1.189000e+01,2.46,2.18,3.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,EarningsPerShareDiluted,NaN,NaN,2.980000e+00,NaN,NaN,NaN,1.05,2.970000e+00,0.61,0.55,0.76,1.25,3.280000e+00,0.64,0.65,0.73,NaN,NaN


In [284]:
income_statements = income_statements_merged.groupby('index').max()

# reindex the merged dataframe using the index of the first dataframe
income_statements = income_statements.reindex(income_statement_2019.index)

# loop over the columns
for col in income_statements.columns[1:]:
    # extract start and end dates from the column label
    splitted = col.split(' to ')
    start = splitted[0]
    end = splitted[1]
        
    # convert start and end dates to datetime objects
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)

    # calculate the duration between start and end dates
    duration = (end_date - start_date).days / 360

    # drop the column if duration is less than a year
    if duration < 1:
        income_statements.drop(columns=[col], inplace=True)

# convert datatype of cells to readable format, e.g. "2.235460e+11" becomes "223546000000"
income_statements = income_statements.apply(lambda row: pd.to_numeric(row, errors='coerce', downcast='integer').fillna('').astype(str), axis=1) 

print("Income statements from Apple's 10-K filings (2016 to 2022) as dataframe")
print('------------------------------------------------------------------------')
income_statements.head(5)

Income statements from Apple's 10-K filings (2016 to 2022) as dataframe
------------------------------------------------------------------------


C:\Users\dotan\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:380: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)


,2016-09-25 to 2017-09-30,2017-10-01 to 2018-09-29,2018-09-30 to 2019-09-28,2019-09-29 to 2020-09-26,2020-09-27 to 2021-09-25,2021-09-26 to 2022-09-24
RevenueFromContractWithCustomerExcludingAssessedTax,229234000000,265595000000,260174000000,274515000000,365817000000,394328000000
CostOfGoodsAndServicesSold,141048000000,163756000000,161782000000,169559000000,212981000000,223546000000
GrossProfit,88186000000,101839000000,98392000000,104956000000,152836000000,170782000000
ResearchAndDevelopmentExpense,11581000000,14236000000,16217000000,18752000000,21914000000,26251000000
SellingGeneralAndAdministrativeExpense,15261000000,16705000000,18245000000,19916000000,21973000000,25094000000
OperatingExpenses,26842000000,30941000000,34462000000,38668000000,43887000000,51345000000
OperatingIncomeLoss,61344000000,70898000000,63930000000,66288000000,108949000000,119437000000
NonoperatingIncomeExpense,2745000000,2005000000,1807000000,803000000,258000000,-334000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,64089000000,72903000000,65737000000,67091000000,109207000000,119103000000
IncomeTaxExpenseBenefit,15738000000,13372000000,10481000000,9680000000,14527000000,19300000000


In [285]:
# convert the columns name into a more readable syntex
income_statements = change_date_column_format(income_statements)

In [286]:
income_statements.head(5)

,"Years ended - September 30, 2017","Years ended - September 29, 2018","Years ended - September 28, 2019","Years ended - September 26, 2020","Years ended - September 25, 2021","Years ended - September 24, 2022"
RevenueFromContractWithCustomerExcludingAssessedTax,229234000000,265595000000,260174000000,274515000000,365817000000,394328000000
CostOfGoodsAndServicesSold,141048000000,163756000000,161782000000,169559000000,212981000000,223546000000
GrossProfit,88186000000,101839000000,98392000000,104956000000,152836000000,170782000000
ResearchAndDevelopmentExpense,11581000000,14236000000,16217000000,18752000000,21914000000,26251000000
SellingGeneralAndAdministrativeExpense,15261000000,16705000000,18245000000,19916000000,21973000000,25094000000
OperatingExpenses,26842000000,30941000000,34462000000,38668000000,43887000000,51345000000
OperatingIncomeLoss,61344000000,70898000000,63930000000,66288000000,108949000000,119437000000
NonoperatingIncomeExpense,2745000000,2005000000,1807000000,803000000,258000000,-334000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,64089000000,72903000000,65737000000,67091000000,109207000000,119103000000
IncomeTaxExpenseBenefit,15738000000,13372000000,10481000000,9680000000,14527000000,19300000000


# Export Income Statement from SEC 10-K Filing to Excel

In [30]:
url_10k_aapl = 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm'

xbrl_json_aapl = xbrlApi.xbrl_to_json(htm_url=url_10k_aapl)

income_statement_aapl = get_income_statement(xbrl_json_aapl)

income_statement_aapl.to_excel('income-statement-aapl.xlsx')

# Getting income statements from more than one company

each company has its own CIK, we need to fetch them and match the stock laebls to thier CIK value 

#### This part includes allot of testing and trying different ways to get to the solution , In the end there is the final code

##### sec-api-documentation link

https://www.sec.gov/edgar/sec-api-documentation

In [237]:
# create request header
headers = {'User-Agent': 'frenkeldotan1998@gmail.com'}

# Get all comapnies data
companyTickers = requests.get('https://www.sec.gov/files/company_tickers.json',headers=headers)

In [238]:
# list of the companies stocks we want to acquire
wish_list = ['AAPL','MSFT','GOOGL']

In [ ]:
companyTickers.json()

In [228]:
# Dictionary to dataframe CIK
companyData = pd.DataFrame.from_dict(companyTickers.json(),orient='index')

In [229]:
# Add missing zeroes to CIK
companyData['cik_str'] = companyData['cik_str'].astype(str).str.zfill(10)

In [230]:
companyData

,cik_str,ticker,title
0,0000320193,AAPL,Apple Inc.
1,0000789019,MSFT,MICROSOFT CORP
2,0001652044,GOOGL,Alphabet Inc.
3,0001018724,AMZN,AMAZON COM INC
4,0001045810,NVDA,NVIDIA CORP
...,...,...,...
9236,0001849280,GSQB-WT,"G Squared Ascend II, Inc."
9237,0001735707,GTXAP,Garrett Motion Inc.
9238,0001813914,CMAXW,"CareMax, Inc."
9239,0001867287,LBBBW,Lakeshore Acquisition II Corp.


In [231]:
# Create a dictionary with the 
dict_cik = {}
for index, row in companyData.iterrows():
    if row['ticker'] in wish_list:
        dict_cik[row['ticker']] = row['cik_str']

In [232]:
dict_cik

{'AAPL': '0000320193', 'MSFT': '0000789019', 'GOOGL': '0001652044'}

In [233]:
# Get the first cik of the data frame - this is the company data we will work on
cik = dict_cik['AAPL']
cik

'0000320193'

In [234]:
# Get company specific metadata and info
fillingMetadata = requests.get(f"https://data.sec.gov/submissions/CIK{cik}.json",headers=headers)

In [ ]:
fillingMetadata.json()

In [ ]:
fillingMetadata.json()['filings']['recent']

In [ ]:
allForms = pd.DataFrame.from_dict(fillingMetadata.json()['filings']['recent'])

In [145]:
# filter only Annual reports
allForms[allForms.form == "10-K"]

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
56,0000320193-22-000108,2022-10-28,2022-09-24,2022-10-27T18:01:14.000Z,34,10-K,001-36743,221338448,,10332356,1,1,aapl-20220924.htm,10-K
131,0000320193-21-000105,2021-10-29,2021-09-25,2021-10-28T18:04:28.000Z,34,10-K,001-36743,211359752,,10502096,1,1,aapl-20210925.htm,10-K
200,0000320193-20-000096,2020-10-30,2020-09-26,2020-10-29T18:06:25.000Z,34,10-K,001-36743,201273977,,12502600,1,1,aapl-20200926.htm,10-K
271,0000320193-19-000119,2019-10-31,2019-09-28,2019-10-30T18:12:36.000Z,34,10-K,001-36743,191181423,,12861616,1,1,a10-k20199282019.htm,10-K
341,0000320193-18-000145,2018-11-05,2018-09-29,2018-11-05T08:01:40.000Z,34,10-K,001-36743,181158788,,12275572,1,0,a10-k20189292018.htm,10-K
429,0000320193-17-000070,2017-11-03,2017-09-30,2017-11-03T08:01:37.000Z,34,10-K,001-36743,171174673,,14071062,1,0,a10-k20179302017.htm,10-K
574,0001628280-16-020309,2016-10-26,2016-09-24,2016-10-26T16:42:16.000Z,34,10-K,001-36743,161953070,,13277662,1,0,a201610-k9242016.htm,10-K
691,0001193125-15-356351,2015-10-28,2015-09-26,2015-10-28T16:31:09.000Z,34,10-K,001-36743,151180619,,9594425,1,0,d17062d10k.htm,FORM 10-K
820,0001193125-14-383437,2014-10-27,2014-09-27,2014-10-27T17:11:55.000Z,34,10-K,000-10030,141175110,,12082626,1,0,d783162d10k.htm,10-K
929,0001193125-13-416534,2013-10-30,2013-09-28,2013-10-29T20:38:28.000Z,34,10-K,000-10030,131177575,,11043722,1,0,d590790d10k.htm,FORM 10-K


In [161]:
full_document = pd.DataFrame(columns=('accessionNumber','primaryDocument'))

In [162]:
full_document['accessionNumber'] = allForms[allForms.form == "10-K"].accessionNumber
full_document['primaryDocument'] = allForms[allForms.form == "10-K"].primaryDocument

In [163]:
full_document

,accessionNumber,primaryDocument
56,0000320193-22-000108,aapl-20220924.htm
131,0000320193-21-000105,aapl-20210925.htm
200,0000320193-20-000096,aapl-20200926.htm
271,0000320193-19-000119,a10-k20199282019.htm
341,0000320193-18-000145,a10-k20189292018.htm
429,0000320193-17-000070,a10-k20179302017.htm
574,0001628280-16-020309,a201610-k9242016.htm
691,0001193125-15-356351,d17062d10k.htm
820,0001193125-14-383437,d783162d10k.htm
929,0001193125-13-416534,d590790d10k.htm


In [206]:
accession_number = full_document.iloc[0][0].replace('-',"")

In [207]:
accession_number

'000032019322000108'

In [208]:
primary_document = full_document.iloc[0][1]

In [209]:
primary_document

'aapl-20220924.htm'

In [179]:
statement_url = f'https://www.sec.gov/ix?doc=/Archives/edgar/data/{cik}/{accession_number}/{primary_document}'

In [181]:
xbrl_json_test = xbrlApi.xbrl_to_json(htm_url=statement_url)

In [182]:
income_statement_test = get_income_statement(xbrl_json_test)


In [183]:
income_statement_test

,2021-09-26 to 2022-09-24,2020-09-27 to 2021-09-25,2019-09-29 to 2020-09-26
RevenueFromContractWithCustomerExcludingAssessedTax,394328000000,365817000000,274515000000
CostOfGoodsAndServicesSold,223546000000,212981000000,169559000000
GrossProfit,170782000000,152836000000,104956000000
ResearchAndDevelopmentExpense,26251000000,21914000000,18752000000
SellingGeneralAndAdministrativeExpense,25094000000,21973000000,19916000000
OperatingExpenses,51345000000,43887000000,38668000000
OperatingIncomeLoss,119437000000,108949000000,66288000000
NonoperatingIncomeExpense,-334000000,258000000,803000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,119103000000,109207000000,67091000000
IncomeTaxExpenseBenefit,19300000000,14527000000,9680000000


# -------------------------------------------------- Final Code --------------------------------------------

In [ ]:
from sec_api import XbrlApi
import requests

In [ ]:
API_KEY = "6ade3b2f0a2495cf4324e580c9ca163f890a2f2d47599ec28ff57bfd4c41cf2e"

xbrlApi = XbrlApi(API_KEY)

### Function
##### Change the index into 10-k syntex

In [ ]:
def change_index_human_readable(income_statement):
    
    human_readable_filing = {'RevenueFromContractWithCustomerExcludingAssessedTax': 'Total net sales',
     'CostOfGoodsAndServicesSold': 'Total cost of sales',
     'GrossProfit': 'Gross margin',
     'ResearchAndDevelopmentExpense': 'Research and development',
     'SellingGeneralAndAdministrativeExpense': 'Selling, general and administrative',
     'OperatingExpenses': 'Total operating expenses',
     'OperatingIncomeLoss': 'Operating income',
     'NonoperatingIncomeExpense': 'Other income/(expense), net',
     'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest': 'Income before provision for income taxes',
     'IncomeTaxExpenseBenefit': 'Provision for income taxes',
     'NetIncomeLoss': 'Net income',
     'EarningsPerShareBasic': 'Earnings per share: Basic',
     'EarningsPerShareDiluted': 'Earnings per share: Diluted',
     'WeightedAverageNumberOfSharesOutstandingBasic': 'Shares used in computing earnings per share: Basic',
     'WeightedAverageNumberOfDilutedSharesOutstanding': 'Shares used in computing earnings per share: Diluted'}

    for index, row in income_statement.iterrows():
        desired_index = index
        new_index_name = human_readable_filing[index]

        income_statement = income_statement.rename(index={desired_index: new_index_name})
        
    return income_statement

### Function
##### Change the column date labels into 10-k format (more easy to read)

In [ ]:
def change_date_column_format(income_statement):
    
    date_lst = []
    for i in income_statement.columns:
        temp = i.split()
        start = pd.to_datetime(temp[0]).strftime("%B %d, %Y")
        end = pd.to_datetime(temp[2]).strftime("%B %d, %Y")
        date_lst.append([start, end])
    
    for i in range(len(income_statement.columns)):
        end_date = date_lst[i][1]
        end_date_str = end_date[0] if isinstance(end_date, list) else end_date
        income_statement = income_statement.rename(columns={income_statement.columns[i]: f'Years ended - {end_date_str}'})
    
    return income_statement

In [ ]:
# create request header
headers = {'User-Agent': 'frenkeldotan1998@gmail.com'}

# Get all comapnies data
companyTickers = requests.get('https://www.sec.gov/files/company_tickers.json',headers=headers)

# Dictionary to dataframe CIK
companyData = pd.DataFrame.from_dict(companyTickers.json(),orient='index')

# Add missing zeroes to CIK
companyData['cik_str'] = companyData['cik_str'].astype(str).str.zfill(10)

# list of the companies stocks we want to acquire
stocks_list = ['AAPL','MSFT','AMZN']
stocks_dataframes_dict = {'APPL': '', 'MSFT': '', 'AMZN': ''} # the keys needs to be equal to the stocks list you wish to find.

# Create a dictionary with the 
dict_cik = {}
for index, row in companyData.iterrows():
    if row['ticker'] in wish_list:
        dict_cik[row['ticker']] = row['cik_str']

        
for company in stocks_list:
    
    # Get the first cik of the data frame - this is the company data we will work on
    cik = dict_cik[company]
    
    # Get company specific metadata and info
    fillingMetadata = requests.get(f"https://data.sec.gov/submissions/CIK{cik}.json",headers=headers)
    
    allForms = pd.DataFrame.from_dict(fillingMetadata.json()['filings']['recent'])
    
    full_document = pd.DataFrame(columns=('accessionNumber','primaryDocument'))
    full_document['accessionNumber'] = allForms[allForms.form == "10-K"].accessionNumber
    full_document['primaryDocument'] = allForms[allForms.form == "10-K"].primaryDocument
    
    income_statment_list = []
    
    for index, row in full_document.iterrows():
        
        accession_number = full_document.loc[index, 'accessionNumber'].replace('-',"")
        primary_document = full_document.loc[index, 'primaryDocument']

        statement_url = f'https://www.sec.gov/ix?doc=/Archives/edgar/data/{cik}/{accession_number}/{primary_document}'

        xbrl_json_temp = xbrlApi.xbrl_to_json(htm_url=statement_url)
        
        income_statment_list.append(get_income_statement(xbrl_json_temp))
        
    income_statements_merged = pd.concat(income_statment_list, axis=0, sort=False)
    
    income_statements = income_statements_merged.groupby('index').max()

    # reindex the merged dataframe using the index of the first dataframe
    income_statements = income_statements.reindex(income_statment_list[0].index)

    # loop over the columns
    for col in income_statements.columns[1:]:
        # extract start and end dates from the column label
        splitted = col.split(' to ')
        start = splitted[0]
        end = splitted[1]

        # convert start and end dates to datetime objects
        start_date = pd.to_datetime(start)
        end_date = pd.to_datetime(end)

        # calculate the duration between start and end dates
        duration = (end_date - start_date).days / 360

        # drop the column if duration is less than a year
        if duration < 1:
            income_statements.drop(columns=[col], inplace=True)

    # convert datatype of cells to readable format, e.g. "2.235460e+11" becomes "223546000000"
    income_statements = income_statements.apply(lambda row: pd.to_numeric(row, errors='coerce', downcast='integer').fillna('').astype(str), axis=1)
    
    # convert the columns name into a more readable syntex
    income_statements = change_date_column_format(income_statements) # for the date values
    income_statements = change_date_column_format(income_statements) # for the index value such as net income, revenues ..
    
    stocks_dataframes_dict[company] = income_statements